In [ ]:
# default_exp funcs.merfish_comparison

In [ ]:
%load_ext autoreload
%autoreload 2
%aimport -decode_fish.engine.place_psfs

In [ ]:
# export
from decode_fish.imports import *

# os.environ["CUDA_VISIBLE_DEVICES"] = '1'

from decode_fish.funcs.file_io import *
from decode_fish.funcs.emitter_io import *
from decode_fish.funcs.utils import *
from decode_fish.funcs.dataset import *
from decode_fish.funcs.plotting import *
from decode_fish.engine.noise import estimate_noise_scale
import shutil
from decode_fish.funcs.visualization import *
from decode_fish.funcs.predict import window_predict
from decode_fish.funcs.predict import *
from decode_fish.funcs.evaluation import *

from omegaconf import open_dict
from hydra import compose, initialize
from decode_fish.funcs.merfish_eval import *

sys.path.append('/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/FQ/istdeco/')

from istdeco import ISTDeco
from utils import random_codebook, random_image_stack
from codebook import Codebook
# from starfish.image import Filter

sys.path.append('/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/FQ/bardensr/')
# import bardensr # Crashes kernel?

import optuna
optuna.logging.set_verbosity(optuna.logging.INFO)

In [ ]:
cfg = OmegaConf.load(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_sim_2D/sim_2d_1/seed:0//train.yaml')

In [ ]:
cfg.genm.prob_generator

{'low': 0.0002, 'high': 0.1}

In [ ]:
codebook, targets = hydra.utils.instantiate(cfg.codebook)

In [ ]:
crop_sz = 128
cfg.training.bs = 5
cfg.genm.PSF.n_cols = 1
cfg.sim.random_crop.crop_sz = crop_sz
cfg.data_path.image_shape = [1,16,1,crop_sz,crop_sz]

model, post_proc, micro, img_3d, decode_dl = load_all(cfg)
psf, noise, micro = load_psf_noise_micro(cfg)

/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.8/site-packages/torch/cuda/__init__.py:122: UserWarning: 
    Found GPU2 NVS 510 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 3.7.
    
  warnings.warn(old_gpu_warn % (d, name, major, minor, min_arch // 10, min_arch % 10))


Crop size larger than volume in at least one dimension. Crop size changed to (1, 48, 48)
1 volumes


In [ ]:
from decode_fish.funcs.routines import *
xsim, cols, gt = sim_data(decode_dl, micro, 10, 
                          int_conc=model.int_dist.int_conc.detach(), int_rate=model.int_dist.int_rate.detach(), int_loc=model.int_dist.int_loc.detach(),
                          channels=micro.noise.channels, n_bits=4, codebook=torch.tensor(codebook), rate_fac=1.5, 
                          pos_noise_xy=cfg.genm.pos_noise.pos_noise_xy, pos_noise_z=cfg.genm.pos_noise.pos_noise_z, 
                          emitter_noise_rate_fac=cfg.genm.emitter_noise.rate_fac, emitter_noise_int_fac=cfg.genm.emitter_noise.int_fac
                         )

/groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
gt = px_to_nm(gt, cfg.evaluation.px_size_zyx)

In [ ]:
pred_dfs = []
net_inp = xsim
for i in range(len(net_inp)):
    pred_df = get_prediction(model, net_inp[i:i+1], post_proc, cuda=True)
    pred_dfs.append(pred_df)

pred_df = cat_emitter_dfs(pred_dfs)

In [ ]:
sub_df = percentile_filter(pred_df, 100, 'comb_sig')
perf, matches, _  = matching(gt, sub_df, match_genes=False, tolerance=300)
print(f'-------', sum(matches['code_inds_pred'] == matches['code_inds_tar'])/len(matches))
perf, matches, _  = matching(gt, sub_df, match_genes=True, tolerance=300)

Recall: 0.916
Precision: 0.985
Jaccard: 90.395
RMSE_vol: 25.004
Eff_3d: 84.234
FN: 707.0 FP: 115.0
Num. matches: 7736.000
Shift: 0.10,-0.31,0.00
------- 0.9701396070320579
Recall: 0.898
Precision: 0.966
Jaccard: 87.115
RMSE_vol: 17.520
Eff_3d: 84.419
FN: 857.0 FP: 265.0
Num. matches: 7586.000
Shift: 0.35,-0.30,0.00


In [ ]:
# export
def get_istdeco_df(sim_data, codebook, psf_sig=(1.7, 1.7), n_iter=100, bg=100.):
    
    istd_results = pd.DataFrame()
    
    n_rounds = codebook.shape[1]
    n_cols = codebook.shape[2]
    
    for i in range(len(sim_data)):

        image_data = np.array(sim_data)[i,:,0]
        image_data = image_data.reshape([n_rounds,n_cols,image_data.shape[-2],image_data.shape[-1]], order='F')
        
        init_th = np.percentile(image_data,50)

        istdeco_model = ISTDeco(image_data, codebook, psf_sig, b=bg)
        X, Q, loss = istdeco_model.run(niter=n_iter)

        # Get codes
        code_id, y, x = np.where(np.logical_and(X>init_th,Q>0.1))

        intensity = X[code_id, y, x]
        quality = Q[code_id, y, x]

        # Store in dataframe
        df = pd.DataFrame(data={
                'frame_idx': i,
                'x': x.astype('float')+0.5, 
                'y': y.astype('float')+0.5,
                'z': 0.5,
                'intensity': intensity, 
                'quality': quality,
                'code_inds': code_id
        })

        istd_results = pd.concat([istd_results,df]) #istd_results.append(df)

    istd_results = px_to_nm(istd_results)

    return istd_results

In [ ]:
import h5py
with h5py.File('/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/data/merfish_sims/sim_data4/dens_0.1_int_1.0.h5', 'r') as f:

    xsim = f['frames'][()]
    gt = get_df_from_hdf5(f['locations'])

In [ ]:
istd_df = get_istdeco_df(cpu(xsim), codebook.reshape([140,8,2], order='F'), psf_sig=(1.7, 1.7), n_iter=400)
# istd_df = exclude_borders(istd_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])

In [ ]:
istd_df = exclude_borders(istd_df, border_size_zyx=[0,500,500], img_size=[128*100,128*100,128*100])

In [ ]:
sub_df = istd_df[istd_df['quality'] > 0.35]
sub_df = sub_df[sub_df['intensity'] > 4500]

In [ ]:
perf, matches, _  = matching(gt, sub_df, match_genes=False, tolerance=300)
print(f'-------', sum(matches['code_inds_pred'] == matches['code_inds_tar'])/len(matches))
perf, matches, _  = matching(gt, sub_df, match_genes=True, tolerance=300)

Recall: 0.889
Precision: 0.944
Jaccard: 84.498
RMSE_vol: 63.343
Eff_3d: 64.738
FN: 1542.0 FP: 733.0
Num. matches: 12401.000
Shift: -0.32,0.26,0.00
------- 0.789210547536489
Recall: 0.785
Precision: 0.834
Jaccard: 67.909
RMSE_vol: 54.964
Eff_3d: 57.750
FN: 2992.0 FP: 2183.0
Num. matches: 10951.000
Shift: -0.35,0.16,0.00


In [ ]:
# istd_df = get_istdeco_df(cpu(xsim), codebook.reshape([140,8,2], order='F'), psf_sig=(1.7, 1.7), n_iter=400, bg=100.)
# istd_df = exclude_borders(istd_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])

q_max = 0.3*istd_df['quality'].max()
i_max = 0.3*istd_df['intensity'].max()
i_min = istd_df['intensity'].min()

def objective(trial):
    
    n_iter = 400 # trial.suggest_uniform('n_iter', 50, 500)
    psf_s = 1.7 # trial.suggest_uniform('psf_s', 1.2, 1.9)
    
#     istd_df = get_istdeco_df(xsimn, code_ref.reshape([140,8,2], order='F'), psf_sig=(psf_s, psf_s), n_iter=int(n_iter))
#     istd_df = exclude_borders(istd_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])
    
    qual_th = trial.suggest_uniform('qual_th', 0.1, q_max)
    int_th = trial.suggest_uniform('int_th', i_min, i_max)
    
    sub_df = istd_df[(istd_df['intensity'] > int_th) & (istd_df['quality'] > qual_th)]
    perf, matches, _  = matching(gt, sub_df, match_genes=True, print_res=False) 
    
    return -perf['jaccard']

study = optuna.create_study()
study.optimize(objective, n_trials=200)

print(study.best_params)
print('Jaccard ', -study.best_value)

[I 2022-05-24 08:26:24,001] A new study created in memory with name: no-name-fcc9b733-95b5-46dd-a354-bf9934652aa2
[I 2022-05-24 08:26:25,154] Trial 0 finished with value: -0.3918928390286522 and parameters: {'qual_th': 0.7032956651518701, 'int_th': 5535.185244624838}. Best is trial 0 with value: -0.3918928390286522.
[I 2022-05-24 08:26:33,082] Trial 1 finished with value: -0.6835822538079435 and parameters: {'qual_th': 0.26568958489275485, 'int_th': 4182.297206745314}. Best is trial 1 with value: -0.6835822538079435.
[I 2022-05-24 08:26:51,520] Trial 2 finished with value: -0.5115155254158799 and parameters: {'qual_th': 0.12263764318715546, 'int_th': 957.701795404839}. Best is trial 1 with value: -0.6835822538079435.
[I 2022-05-24 08:26:59,271] Trial 3 finished with value: -0.6534094076643718 and parameters: {'qual_th': 0.3356277238214595, 'int_th': 1352.7955997987099}. Best is trial 1 with value: -0.6835822538079435.
[I 2022-05-24 08:26:59,636] Trial 4 finished with value: -0.20109936

[I 2022-05-24 08:30:03,523] Trial 37 finished with value: -0.6747758474698663 and parameters: {'qual_th': 0.14646401581140703, 'int_th': 5044.536651611353}. Best is trial 12 with value: -0.6914239993681778.
[I 2022-05-24 08:30:08,389] Trial 38 finished with value: -0.6354258874139238 and parameters: {'qual_th': 0.43747511864311794, 'int_th': 1995.4755115882622}. Best is trial 12 with value: -0.6914239993681778.
[I 2022-05-24 08:30:12,746] Trial 39 finished with value: -0.637544034483309 and parameters: {'qual_th': 0.21630246025072533, 'int_th': 7631.225389662888}. Best is trial 12 with value: -0.6914239993681778.
[I 2022-05-24 08:30:21,057] Trial 40 finished with value: -0.6479820555158962 and parameters: {'qual_th': 0.32528557616343834, 'int_th': 1119.6227657785394}. Best is trial 12 with value: -0.6914239993681778.
[I 2022-05-24 08:30:27,237] Trial 41 finished with value: -0.6846196472994058 and parameters: {'qual_th': 0.276621420952496, 'int_th': 5789.97127431384}. Best is trial 12 

[I 2022-05-24 08:34:22,751] Trial 75 finished with value: -0.6876688689797485 and parameters: {'qual_th': 0.2827561079478264, 'int_th': 5571.186855783381}. Best is trial 12 with value: -0.6914239993681778.
[I 2022-05-24 08:34:29,233] Trial 76 finished with value: -0.6903324099989946 and parameters: {'qual_th': 0.3058530555695615, 'int_th': 4913.576413289925}. Best is trial 12 with value: -0.6914239993681778.
[I 2022-05-24 08:34:31,576] Trial 77 finished with value: -0.532191490821413 and parameters: {'qual_th': 0.5721482596130454, 'int_th': 4638.032267769391}. Best is trial 12 with value: -0.6914239993681778.
[I 2022-05-24 08:34:39,116] Trial 78 finished with value: -0.6847964101301177 and parameters: {'qual_th': 0.3374692979498041, 'int_th': 3550.7405819565283}. Best is trial 12 with value: -0.6914239993681778.
[I 2022-05-24 08:34:50,664] Trial 79 finished with value: -0.6806146335562305 and parameters: {'qual_th': 0.31102007912014307, 'int_th': 6155.736306484362}. Best is trial 12 wi

[I 2022-05-24 08:38:49,326] Trial 113 finished with value: -0.6885879994126208 and parameters: {'qual_th': 0.3233344170341098, 'int_th': 4053.8231871300477}. Best is trial 12 with value: -0.6914239993681778.
[I 2022-05-24 08:38:56,226] Trial 114 finished with value: -0.6893086975596089 and parameters: {'qual_th': 0.3047737227184369, 'int_th': 4498.864251808227}. Best is trial 12 with value: -0.6914239993681778.
[I 2022-05-24 08:39:03,260] Trial 115 finished with value: -0.6892191443732348 and parameters: {'qual_th': 0.3073754549394651, 'int_th': 4272.4219512612735}. Best is trial 12 with value: -0.6914239993681778.
[I 2022-05-24 08:39:09,877] Trial 116 finished with value: -0.6863603125898332 and parameters: {'qual_th': 0.33803455578672953, 'int_th': 3764.9361951549454}. Best is trial 12 with value: -0.6914239993681778.
[I 2022-05-24 08:39:16,492] Trial 117 finished with value: -0.690598282166077 and parameters: {'qual_th': 0.30244109008996484, 'int_th': 5010.164658268724}. Best is tri

[I 2022-05-24 08:42:58,771] Trial 151 finished with value: -0.6905586881809387 and parameters: {'qual_th': 0.29708177308816164, 'int_th': 4995.234004669268}. Best is trial 133 with value: -0.6921530316540387.
[I 2022-05-24 08:43:05,899] Trial 152 finished with value: -0.6896531089563342 and parameters: {'qual_th': 0.27909985689225975, 'int_th': 4759.954949525897}. Best is trial 133 with value: -0.6921530316540387.
[I 2022-05-24 08:43:12,351] Trial 153 finished with value: -0.6903016893819263 and parameters: {'qual_th': 0.29539890705283933, 'int_th': 5199.114690146232}. Best is trial 133 with value: -0.6921530316540387.
[I 2022-05-24 08:43:15,262] Trial 154 finished with value: -0.5675749056523082 and parameters: {'qual_th': 0.3167672193267297, 'int_th': 9015.413583662039}. Best is trial 133 with value: -0.6921530316540387.
[I 2022-05-24 08:43:15,733] Trial 155 finished with value: -0.2315481675597494 and parameters: {'qual_th': 0.28685886827354146, 'int_th': 15252.278636077954}. Best i

[I 2022-05-24 08:47:06,448] Trial 188 finished with value: -0.6875115891938938 and parameters: {'qual_th': 0.28941108095483586, 'int_th': 5668.207708299887}. Best is trial 133 with value: -0.6921530316540387.
[I 2022-05-24 08:47:12,334] Trial 189 finished with value: -0.6863344266658213 and parameters: {'qual_th': 0.3489320646989066, 'int_th': 4605.284587232626}. Best is trial 133 with value: -0.6921530316540387.
[I 2022-05-24 08:47:18,254] Trial 190 finished with value: -0.6888819401763654 and parameters: {'qual_th': 0.3292308526664647, 'int_th': 5224.044397124246}. Best is trial 133 with value: -0.6921530316540387.
[I 2022-05-24 08:47:24,890] Trial 191 finished with value: -0.6906514118283387 and parameters: {'qual_th': 0.30264700130100136, 'int_th': 4885.731542732349}. Best is trial 133 with value: -0.6921530316540387.
[I 2022-05-24 08:47:31,706] Trial 192 finished with value: -0.6910991517179388 and parameters: {'qual_th': 0.29600628572522975, 'int_th': 4821.418348150219}. Best is 

{'qual_th': 0.31129289116991726, 'int_th': 4710.391866508803}
Jaccard  0.6921530316540387


In [ ]:
sub_df = istd_df[(istd_df['intensity'] > study.best_params['int_th']) & (istd_df['quality'] > study.best_params['qual_th'])]
perf, matches, _  = matching(gt_df, sub_df, match_genes=False)
print(f'-------', sum(matches['code_inds_pred'] == matches['code_inds_tar'])/len(matches))
perf, matches, _  = matching(gt_df, sub_df, match_genes=True)

Recall: 0.595
Precision: 0.729
Jaccard: 48.749
RMSE_vol: 169.596
Eff_3d: 0.918
FN: 649.0 FP: 355.0
Num. matches: 955.000
Shift: 2.48,3.01,0.00
------- 0.5821989528795811
Recall: 0.392
Precision: 0.479
Jaccard: 27.472
RMSE_vol: 50.623
Eff_3d: 23.182
FN: 976.0 FP: 682.0
Num. matches: 628.000
Shift: 1.57,0.01,0.00


In [ ]:
# export
def get_bardensr_tensor(sim_data, codebook, bg=100., n_iter=400, l1_pen=0., psf_r=0):
    
    bard_results = pd.DataFrame()
    evd_tensors = []
    codeflat = codebook.T
    
    for i in range(len(sim_data)):
    
        image_data = sim_data[i]
        Xnorm = image_data - bg
#         Xnorm /= Xnorm.max()
        
#         Xnorm = bardensr.preprocessing.minmax(image_data - bg)

#         Xnorm = bardensr.preprocessing.minmax(image_data)
#         Xnorm = bardensr.preprocessing.background_subtraction(Xnorm,[0,10,10])
        Xnorm = bardensr.preprocessing.minmax(Xnorm)

        evidence_tensor_iterative,extra_learned_params=\
            bardensr.spot_calling.estimate_density_iterative(Xnorm.astype('float64'),codeflat,l1_penalty=l1_pen,use_tqdm_notebook=True,iterations=n_iter, estimate_codebook_gain=True, psf_radius=(psf_r, psf_r, psf_r))

#         evidence_tensor_iterative= bardensr.spot_calling.estimate_density_singleshot(Xnorm.astype('float64'), codeflat, noisefloor=0.05)
    
        evd_tensors.append(evidence_tensor_iterative)
        
    return evd_tensors

def get_bardensr_df(evd_tensors, th, ps=1.0):
    
    bard_results = pd.DataFrame()

    for i in range(len(evd_tensors)):    
    #     thresh_iterative=evd_tensors[i].max()*.1
        result_iterative=bardensr.spot_calling.find_peaks(evd_tensors[i],th, poolsize=(ps,ps,ps))

        code_inds = np.array(result_iterative.j.values, dtype=np.int16)
        df = pd.DataFrame(data={
                    'frame_idx': i,
                    'x': result_iterative.m2.values + 0.5, 
                    'y': result_iterative.m1.values + 0.5,
                    'z': 0.5,
                    'intensity': result_iterative.int.values,
                    'code_inds': code_inds
            })

        bard_results = bard_results.append(df)

    bard_results = px_to_nm(bard_results)
    return bard_results

In [ ]:
evd_tensors = get_bardensr_tensor(cpu(xsimn), code_ref, n_iter=100, l1_pen=.01, bg=100., psf_r=0)

2021-09-25 11:24:06.459223: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-25 11:24:06.460009: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-25 11:24:06.461228: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2021-09-25 11:24:06.461275: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-09-25 11:24:06.461340: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-09-25 11:24:06.461381: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2021-09-25 11

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
bard_df = get_bardensr_df(evd_tensors, th=.25, ps=1.05)
# bard_df = exclude_borders(bard_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])

2021-09-25 11:24:10.373225: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


In [ ]:
len(bard_df)

8159

In [ ]:
perf, matches, _  = matching(gt_df, bard_df, match_genes=False)
print(f'-------', sum(matches['code_inds_pred'] == matches['code_inds_tar'])/len(matches))
perf, matches, _  = matching(gt_df, bard_df, match_genes=True)

Recall: 0.858
Precision: 0.850
Jaccard: 74.517
RMSE_vol: 143.815
Eff_3d: 23.711
FN: 1149.0 FP: 1223.0
Num. matches: 6936.000
Shift: -1.75,2.97,0.00
------- 0.5641580161476355
Recall: 0.623
Precision: 0.618
Jaccard: 44.984
RMSE_vol: 81.173
Eff_3d: 31.633
FN: 3045.0 FP: 3119.0
Num. matches: 5040.000
Shift: 0.37,1.18,0.00


In [ ]:
# evd_tensors = get_bardensr_tensor(xsimn, code_ref, n_iter=int(300), l1_pen=0.01)

def objective(trial):
    
#     n_iter = 300
#     l1_pen = trial.suggest_uniform('l1_pen', 0., 0.1)
    th     = trial.suggest_uniform('th', 0.2, 0.6)
    ps     = trial.suggest_uniform('ps', 0.1, 1.5)
    
#     evd_tensors = get_bardensr_tensor(xsimn, code_ref, n_iter=int(n_iter), l1_pen=l1_pen)
#     print(evd_tensors[0].mean())
    bard_df = get_bardensr_df(evd_tensors, th, ps)
    bard_df = exclude_borders(bard_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])
    
    perf, matches, _  = matching(gt_df, bard_df, match_genes=True, print_res=False) 
    
    return -perf['jaccard']

study = optuna.create_study()
study.optimize(objective, n_trials=200)

print(study.best_params)
print('Jaccard ', -study.best_value)

In [ ]:
evd_tensors = get_bardensr_tensor(cpu(xsimn), code_ref, n_iter=100, l1_pen=.05, bg=100., psf_r=0)

In [ ]:
bard_df = get_bardensr_df(evd_tensors, th=study.best_params['th'], ps=study.best_params['ps'])
# bard_df = get_bardensr_df(evd_tensors, th=0.5, ps=1.05)
bard_df = exclude_borders(bard_df, border_size_zyx=[0,500,500], img_size=[crop_sz*100,crop_sz*100,crop_sz*100])
perf, matches, _  = matching(gt_df, bard_df, match_genes=False)
print(f'-------', sum(matches['code_inds_pred'] == matches['code_inds_tar'])/len(matches))
perf, matches, _  = matching(gt_df, bard_df, match_genes=True)

In [ ]:
!nbdev_build_lib

Converted 00_models.ipynb.
Converted 01_psf.ipynb.
Converted 02_microscope.ipynb.
Converted 03_noise.ipynb.
Converted 04_pointsource.ipynb.
Converted 05_gmm_loss.ipynb.
Converted 06_plotting.ipynb.
Converted 07_file_io.ipynb.
Converted 08_dataset.ipynb.
Converted 09_output_trafo.ipynb.
Converted 10_evaluation.ipynb.
Converted 11_emitter_io.ipynb.
Converted 12_utils.ipynb.
Converted 13_train.ipynb.
Converted 15_fit_psf.ipynb.
Converted 16_visualization.ipynb.
Converted 17_eval_routines.ipynb.
Converted 18_predict_funcs.ipynb.
Converted 19_MERFISH_routines.ipynb.
Converted 20_MERFISH_visualization.ipynb.
Converted 22_MERFISH_codenet.ipynb.
Converted 23_MERFISH_comparison.ipynb.
Converted index.ipynb.
